In [3]:
!pip install BeautifulSoup4
!pip install requests

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

dataSource = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(dataSource.text, 'lxml')

data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())

        if (index == 0):
            columns = section
        else:
            data.append(section)
    
canada_data = pd.DataFrame(data = data,columns = columns)
canada_data = canada_data[canada_data['Borough'] != 'Not assigned']
canada_data["Neighborhood"] = canada_data.groupby("Post Code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))
canada_data = canada_data.drop_duplicates()
if(canada_data.index.name != 'Post Code'):
    canada_data = canada_data.set_index('Post Code')
    
canada_data.head()

,Borough,Neighborhood
Post Code,,
M3A,North York,"Parkwoods, Parkwoods, Parkwoods"
M4A,North York,"Victoria Village, Victoria Village, Victoria V..."
M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou..."
M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man..."
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q..."


In [8]:
import pandas as pd
import io
import requests

url="https://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

canada_Withcoordinates_df= canada_data.join(c.set_index('Postal Code'))
canada_Withcoordinates_df.head(11)

,Borough,Neighborhood,Latitude,Longitude
Post Code,,,,
M3A,North York,"Parkwoods, Parkwoods, Parkwoods",43.753259,-79.329656
M4A,North York,"Victoria Village, Victoria Village, Victoria V...",43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou...",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man...",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q...",43.662301,-79.389494
M9A,Etobicoke,"Islington Avenue, Humber Valley Village, Islin...",43.667856,-79.532242
M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353
M3B,North York,"Don Mills, Don Mills, Don Mills",43.745906,-79.352188
M4B,East York,"Parkview Hill, Woodbine Gardens, Parkview Hill...",43.706397,-79.309937


In [12]:
#Filter Canada data for Toronto
toronto_df = canada_Withcoordinates_df[canada_Withcoordinates_df['Borough'].str.contains('Toronto')]
toronto_df.head()

,Borough,Neighborhood,Latitude,Longitude
Post Code,,,,
M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou...",43.654260,-79.360636
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q...",43.662301,-79.389494
M5B,Downtown Toronto,"Garden District, Ryerson, Garden District, Rye...",43.657162,-79.378937
M5C,Downtown Toronto,"St. James Town, St. James Town, St. James Town",43.651494,-79.375418
M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031


In [10]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium 
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [15]:
toronto_map = folium.Map(location=[toronto_df["Latitude"][0], toronto_df["Longitude"][0]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  

toronto_map
    

In [17]:
CLIENT_ID = 'WJUNQOCHB2RHDPJEKHMEOUC0EDX5TQEKOTPHA2VJKHFJARFI'
CLIENT_SECRET = 'I1WAYHXO1IGDPEDS4KX3S24ZXDO2P4G5FU3IQRMIW1FRZX01' 
VERSION = '20180605'

neighborhood_latitude = toronto_df['Latitude'][0] 
neighborhood_longitude = toronto_df['Longitude'][0] 

neighborhood_name = toronto_df['Neighborhood'][0] 



# Setup API URL to explore venues near by
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
neighborhood_json = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = neighborhood_json['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues_df = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront, Regent Park, Harbourfront, Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government, Queen's Park, Ontario Provincial Government, Queen's Park, Ontario Provincial Government
Garden District, Ryerson, Garden District, Ryerson, Garden District, Ryerson
St. James Town, St. James Town, St. James Town
The Beaches, The Beaches, The Beaches
Berczy Park, Berczy Park, Berczy Park
Central Bay Street, Central Bay Street, Central Bay Street
Christie, Christie, Christie
Richmond, Adelaide, King, Richmond, Adelaide, King, Richmond, Adelaide, King
Dufferin, Dovercourt Village, Dufferin, Dovercourt Village, Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands, Harbourfront East, Union Station, Toronto Islands, Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity, Little Portugal, Trinity, Little Portugal, Trinity
The Danforth West, Riverdale, The Danforth West, Riverdale, The Danforth West, Riverdale
Toronto Domini

In [21]:
print(toronto_venues_df.shape)
toronto_venues_df.head()

(1633, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront, Regent Park, Harbou...",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront, Regent Park, Harbou...",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront, Regent Park, Harbou...",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront, Regent Park, Harbou...",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront, Regent Park, Harbou...",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [22]:
toronto_venues_df.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Berczy Park, Berczy Park, Berczy Park",59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place, Brockton, Parkdale Village, Exhibition Place, Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto, Business reply mail Processing Centre, South Central Letter Processing Plant Toronto, Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport, CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport, CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
"Central Bay Street, Central Bay Street, Central Bay Street",65,65,65,65,65,65
"Christie, Christie, Christie",16,16,16,16,16,16
"Church and Wellesley, Church and Wellesley, Church and Wellesley",74,74,74,74,74,74
"Commerce Court, Victoria Hotel, Commerce Court, Victoria Hotel, Commerce Court, Victoria Hotel",100,100,100,100,100,100
"Davisville North, Davisville North, Davisville North",8,8,8,8,8,8


In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,"Berczy Park, Berczy Park, Berczy Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.016949,0.000000,0.000000,0.000000,0.0000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place, ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
4,"Central Bay Street, Central Bay Street, Centra...",0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.015385,0.000000,0.000000,0.015385,0.0000,0.000000
5,"Christie, Christie, Christie",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
6,"Church and Wellesley, Church and Wellesley, Ch...",0.027027,0.013514,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013514,...,0.013514,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
7,"Commerce Court, Victoria Hotel, Commerce Court...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.0000,0.000000
8,"Davisville North, Davisville North, Davisville...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
9,"Davisville, Davisville, Davisville",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.030303,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000


In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Berczy Park, Berczy Park, Berczy Park",Coffee Shop,Seafood Restaurant,Cheese Shop,Farmers Market,Bakery,Restaurant,Pharmacy,Café,Beer Bar,Cocktail Bar
1,"Brockton, Parkdale Village, Exhibition Place, ...",Café,Breakfast Spot,Nightclub,Coffee Shop,Climbing Gym,Burrito Place,Italian Restaurant,Intersection,Stadium,Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Brewery,Fast Food Restaurant,Farmers Market,Burrito Place,Auto Workshop,Spa,Pizza Place,Comic Shop,Gym / Fitness Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Sculpture Garden,Plane,Harbor / Marina,Boat or Ferry,Airport Terminal,Rental Car Location,Airport Gate,Airport Food Court
4,"Central Bay Street, Central Bay Street, Centra...",Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store,Bar


In [28]:
kclusters = int(len(toronto_df["Neighborhood"].unique()) / 4)
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)


kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)


neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df.copy()
toronto_merged.rename(columns={'Neighborhood':'Neighborhood'}, inplace=True)

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Post Code,,,,,,,,,,,,,,,
M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou...",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Shoe Store
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q...",43.662301,-79.389494,1,Coffee Shop,Diner,College Cafeteria,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park,College Auditorium
M5B,Downtown Toronto,"Garden District, Ryerson, Garden District, Rye...",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Bubble Tea Shop,Hotel,Cosmetics Shop,Japanese Restaurant,Café,Italian Restaurant,Ramen Restaurant,Pizza Place
M5C,Downtown Toronto,"St. James Town, St. James Town, St. James Town",43.651494,-79.375418,1,Café,Coffee Shop,Restaurant,Clothing Store,American Restaurant,Cosmetics Shop,Cocktail Bar,Seafood Restaurant,Italian Restaurant,Gym
M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031,6,Asian Restaurant,Pub,Trail,Health Food Store,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Women's Store


In [29]:
map_clusters = folium.Map(location=[toronto_df["Latitude"][0], toronto_df["Longitude"][0]], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters